# TruBot testing

This notebook tests a conversation bot with vector-store context of TruEra website. 

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
from trulens_eval.keys import *
from trulens_eval.slackbot import get_or_make_chain, get_answer
from trulens_eval.util import TP
from trulens_eval import Tru
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.util import instrumented_classes, jsonify
from trulens_eval.util import Obj, ObjSerial, Method, FunctionOrMethod, all_objects, Class
import json
import pydantic

from pprint import PrettyPrinter
pp = PrettyPrinter()

Tru().reset_database()

In [ ]:
chain = get_or_make_chain(cid=None)

In [ ]:
# Enumerate instrumented components:

from trulens_eval.utils.langchain import Is

tests = [Is.chain, Is.vector_store, Is.llm, Is.prompt, Is.retriever, Is.memory, Is.chathistory]

for q, ci, obj in instrumented_classes(jsonify(chain)):

    assert isinstance(ci, Class), f"class_info was not a Class object, was {ci}, {type(ci)}."

    print(q, list(Is.what(ci)))

In [ ]:
proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

In [ ]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

selectors = selectors[2:3]
messages = messages[2:3]

def test_bot(selector, question):
    print(selector, question)
    chain = get_or_make_chain(cid=question + str(selector), selector=selector)
    answer = get_answer(chain=chain, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

TP().finish()

# Local Model Testing

In [ ]:
from langchain import ConversationChain, HuggingFacePipeline
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native", device_map='auto')

# Convert the model's parameters to a higher precision data type
base_model = base_model.float()

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# We are going to set the memory to go back 4 turns
# window_memory = ConversationBufferWindowMemory(k=4)

In [ ]:
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True
)